All SAW starts, by convention, at (0,0) and perform a first move to the right.
The possible actions are 1 (right), 2 (up), 3 (left), 4 (down)

In [1]:
from random import randint

In [21]:
def SpecialCases(Walk):
    
    ''' This function detects three types of special cases in the construction of a SAW
    0. Collision: when the position (X,Y) are already part of the Walk.
    1. Contact: when the position (X,Y) is in contact with at least one step of the SAW
    2. Corner: when the position (X,Y) is in contact with a step with one corner
    The first output is 0,1,2 for the three cases mentioned above
    The second ouput is the step for the collision, contact or corner.
    '''
    
    WalkLength = len(Walk)    #length of the walk, ie nbr of steps
    CurrentStep = 0           #we start at the beginning of the walk
    ReachedEnd = False
    OutputDistance = WalkLength #The first output is given an initial value
    OutputStep = WalkLength     #The second output is given an initial value
    FirstCorner = WalkLength
    FirstContact = WalkLength
    
    X = Walk[WalkLength-1][0]
    Y = Walk[WalkLength-1][1]
    #print(X,Y)
    #We loop until the end is reached or a collision takes place
    while not ReachedEnd:

        #We recompute the distance for the next loop
        DistanceL2 = (Walk[CurrentStep][0]-X)**2+(Walk[CurrentStep][1]-Y)**2
        DistanceL1 = abs(Walk[CurrentStep][0]-X)+abs(Walk[CurrentStep][1]-Y)
        StepIncrement = DistanceL1 - 2
        #print("Distance: ", DistanceL2)
        #print("Progress: ", DistanceL1)
        
        #Collision: return 0 and the location of the collision in the walk (stop searching)
        if DistanceL2 == 0: return DistanceL2,CurrentStep

        #Contact: return 1 and the location of the contact (keep searching for collision)
        if DistanceL2 == 1 :
            #We only want to have the first contact
            if CurrentStep < FirstContact:
                FirstContact = CurrentStep
                OutputStep = CurrentStep
                OutputDistance = DistanceL2
            StepIncrement += 2

        #Corner: return 2 and the location of the corner (keep searching for collision or contact)
        if DistanceL2 == 2 and OutputDistance > 1:
            #We only want to have the first corner
            if CurrentStep < FirstCorner:
                FirstCorner = CurrentStep
                OutputStep = CurrentStep
                OutputDistance = DistanceL2
        
        #Special case: two steps apart but in the same direction
        if DistanceL1 == 2: StepIncrement += 1
            
        #Move the current step by the distance minus 1 (to find when the distance is 1)
        CurrentStep += StepIncrement   
        #print("CurrentStep: ",CurrentStep)
        
        #test if we reach the end of the walk
        if CurrentStep >= (WalkLength-3):
            ReachedEnd = True

    return OutputDistance,OutputStep

In [22]:
SAW = [(0, 0), (1, 0), (2, 0), (3, 0), (3, -1), (2, -1), (1, -1), (1, -2), (0, -2), (-1, -2), (-1, -3), (-2, -3), (-3, -3), (-3, -2), (-2, -2)]
print(SpecialCases(SAW))

(1, 9)


In [23]:
def OneStepWithoutReverse(Walk,LastAction):
    AllAction = [1,2,3,4]
    OppositeAction = [3,4,1,2]

    PossibleNextAction= AllAction
    PossibleNextAction.remove(OppositeAction[LastAction-1])
    #and select the next action randomly
    NextAction = PossibleNextAction[randint(1,len(PossibleNextAction))-1]

    AddStep(Walk,NextAction)
  
    return NextAction

def AddStep(Walk,Action):
    
    WalkLenght = len(Walk)
    CurrentX = Walk[WalkLenght-1][0]
    CurrentY = Walk[WalkLenght-1][1]
    
    #We move the position
    if Action == 1:
        NewX = CurrentX + 1
        NewY = CurrentY
    if Action == 2:
        NewX = CurrentX
        NewY = CurrentY + 1
    if Action == 3:
        NewX = CurrentX - 1
        NewY = CurrentY
    if Action == 4:
        NewX = CurrentX
        NewY = CurrentY - 1
    Walk.append((NewX,NewY))
    return _

def WhichAction(deltaX,deltaY):
    output = -1
    if deltaX == 1 and deltaY == 0: output = 1
    if deltaX == 0 and deltaY == 1: output = 2
    if deltaX == -1 and deltaY == 0: output = 3
    if deltaX == 0 and deltaY == -1: output = 4
    return output
        

def OneStepWithContact(Walk,LastAction,ContactStep):
    
    WalkLength = len(Walk)
    
    AllAction = [1,2,3,4]
    OppositeAction = [3,4,1,2]

    PossibleNextAction= AllAction
    PossibleNextAction.remove(OppositeAction[LastAction-1])
    
    #we need first to remove the potential collision
    CurrentX = Walk[WalkLength-1][0]
    CurrentY = Walk[WalkLength-1][1]
    ContactX = Walk[ContactStep][0]
    ContactY = Walk[ContactStep][1]
    MoveX = (ContactX - CurrentX)
    MoveY = (ContactY - CurrentY)

    #Identify the action taken based on the move
    CollisionAction = WhichAction(MoveX,MoveY)

    PossibleNextAction.remove(CollisionAction)

    #next, we need to remove going in the same direction than the flow
    ContactXMO = Walk[ContactStep-1][0]
    ContactYMO = Walk[ContactStep-1][1]
    MoveX = (ContactX - ContactXMO)
    MoveY = (ContactY - ContactYMO)

    #Identify the action taken based on the move
    FirstFlowAction = WhichAction(MoveX,MoveY)

    if FirstFlowAction in PossibleNextAction:
        PossibleNextAction.remove(FirstFlowAction)
        
    #next, remove going in the same directon than the flow after the contact

    ContactXPO = Walk[ContactStep+1][0]
    ContactYPO = Walk[ContactStep+1][1]
    MoveX = (ContactXPO - ContactX)
    MoveY = (ContactYPO - ContactY)

    #Identify the action taken based on the move
    SecondFlowAction = WhichAction(MoveX,MoveY)

    if SecondFlowAction in PossibleNextAction:
        PossibleNextAction.remove(SecondFlowAction)

    #and select the next action randomly
    NextAction = PossibleNextAction[randint(1,len(PossibleNextAction))-1]

    AddStep(Walk,NextAction)
    
    return NextAction

def OneStepWithCorner(Walk,LastAction,ContactStep):
    
    AllAction = [1,2,3,4]
    OppositeAction = [3,4,1,2]

    PossibleNextAction= AllAction
    PossibleNextAction.remove(OppositeAction[LastAction-1])
    
    ContactX = Walk[ContactStep][0]
    ContactY = Walk[ContactStep][1]

    #we want to remove the action going in the direction of the walk
    #next, we need to remove going in the same direction than the flow
    ContactXMO = Walk[ContactStep-1][0]
    ContactYMO = Walk[ContactStep-1][1]
    MoveX = (ContactX - ContactXMO)
    MoveY = (ContactY - ContactYMO)

    #Identify the action taken based on the move
    FirstFlowAction = WhichAction(MoveX,MoveY)

    if FirstFlowAction in PossibleNextAction:
        PossibleNextAction.remove(FirstFlowAction)
        
    #next, remove going in the same directon than the flow after the contact

    ContactXPO = Walk[ContactStep+1][0]
    ContactYPO = Walk[ContactStep+1][1]
    MoveX = (ContactXPO - ContactX)
    MoveY = (ContactYPO - ContactY)

    #Identify the action taken based on the move
    SecondFlowAction = WhichAction(MoveX,MoveY)

    if SecondFlowAction in PossibleNextAction:
        PossibleNextAction.remove(SecondFlowAction)

    
    #and select the next action randomly
    NextAction = PossibleNextAction[randint(1,len(PossibleNextAction))-1]

    AddStep(Walk,NextAction)

    return NextAction

In [24]:
#Starting point is (0,0)
SAW = [(0,0)]

#By default we start by increasing x by one
SAW.append((1,0))
LastAction = 1

#The next two steps can be performed savely as there is no risk of collision
LastAction = OneStepWithoutReverse(SAW,LastAction)
LastAction = OneStepWithoutReverse(SAW,LastAction)

print(SAW)

for _ in range(20):

#Before additing it to the walk, we need to check if there is a collision
    SpecialCase, CaseStep = SpecialCases(SAW)
    #print("Special case: ", SpecialCase)
    if SpecialCase == 0:
        # there is a collision
        print("Algorithm is wrong!")

    if SpecialCase == 1:
        #contact:
        print("Contact")
        LastAction = OneStepWithContact(SAW,LastAction,CaseStep)


    if SpecialCase == 2:
        #corner
        print("Corner")
        LastAction = OneStepWithCorner(SAW,LastAction,CaseStep)

    
    if SpecialCase > 2:
        LastAction = OneStepWithoutReverse(SAW,LastAction)
    
    last = len(SAW)-1
    print(SAW[last][0],SAW[last][1])
    

[(0, 0), (1, 0), (2, 0), (2, 1)]
2 2
3 2
4 2
4 1
5 1
5 2
Contact
5 3
Corner
5 4
5 5
4 5
3 5
3 4
4 4
Contact
4 3
Contact
3 3
Contact
2 3
Contact
1 3
Corner
0 3
-1 3
-1 4


In [25]:
print(SAW)

[(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (3, 2), (4, 2), (4, 1), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (4, 5), (3, 5), (3, 4), (4, 4), (4, 3), (3, 3), (2, 3), (1, 3), (0, 3), (-1, 3), (-1, 4)]
